In [1]:
# # !pip install git+https://g+ithub.com/kunaldahiya/pyxclib.git
# # Clone the repo
# !git clone https://github.com/kunaldahiya/pyxclib.git

# # Go inside and install (ignoring C++ parts)
# %cd pyxclib

# # Remove the C++ extensions from setup.py (temporary fix)
# !sed -i '/ext_modules/d' setup.py

# # Install the pure Python parts
# !pip install .






# !git clone https://github.com/kunaldahiya/pyxclib.git

# # Change directory and install
# %cd pyxclib
# !pip install .





# git clone https://github.com/kunaldahiya/pyxclib.git
# cd pyxclib
# !python3 setup.py install --user

In [2]:
def load_data(path):
    texts = []
    labels = []
    with open(path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if not parts:
                continue

            # Check if label part contains only digits and commas
            label_candidate = parts[0]
            if ':' in label_candidate:
                # No labels — malformed line — skip
                continue

            try:
                label_list = list(map(int, label_candidate.split(',')))
            except ValueError:
                continue  # Skip bad label formats

            feature_strs = parts[1:]
            feature_tokens = [fs.split(':')[0] for fs in feature_strs]
            texts.append(' '.join(feature_tokens))
            labels.append(label_list)

    return texts, labels

# Load data
train_texts, train_labels = load_data('/kaggle/input/eurlex/train.txt')
test_texts, test_labels = load_data('/kaggle/input/eurlex/test.txt')



In [3]:
print(train_texts[0])   # '0 1 2 3 4 5 6 ...'
print(train_labels[0])  # [446, 521, 1149, 1249, ...]

0 1 2 3 4 5 6 7 8 10 11 12 19 20 24 26 27 29 32 33 34 38 42 44 60 64 74 75 90 91 108 115 141 163 174 190 291 300 323 425 450 484 490 583 652 684 766 853 928 1173 1184 1269 4277
[446, 521, 1149, 1249, 1265, 1482]


In [4]:
# !head -n 5 /kaggle/input/eurlex/train.txt

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=250000)
X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)

In [6]:
from sklearn.datasets import dump_svmlight_file

def write_dismec_format(X, y, path):
    y_str = [' '.join(str(label) for label in labels) for labels in y]
    with open(path, 'w') as f:
        for i in range(X.shape[0]):
            row = X.getrow(i)
            s = y_str[i] + " "
            s += ' '.join([f"{j+1}:{v:.6f}" for j, v in zip(row.indices, row.data)])
            f.write(s + "\n")

write_dismec_format(X_train, train_labels, 'train-remapped-tfidf-relabeled.txt')
write_dismec_format(X_test, test_labels, 'test-remapped-tfidf-relabeled.txt')

In [7]:
write_dismec_format(X_train, train_labels, '/kaggle/working/train-remapped-tfidf-relabeled.txt')
write_dismec_format(X_test, test_labels, '/kaggle/working/test-remapped-tfidf-relabeled.txt')
